In [3]:

import keras
from keras import layers
from keras import ops
import tensorflow as tf
import os
import numpy as np
from glob import glob
import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm  
# For data preprocessing
from keras import losses
from keras.losses import binary_crossentropy
from PIL import Image
from tensorflow import image as tf_image
from tensorflow import data as tf_data
from tensorflow import io as tf_io
from keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# (2) keras backend 문법 사용 가능

def dice_score_loss(y_true, y_pred):
  numerator = 2. * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)
  return tf.reduce_mean(1 - numerator / denominator)





In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 8
NUM_CLASSES = 4
train_df = pd.read_csv(
    '../../data/segmentationDDH/train_aug_segmentation_dataset.csv')
val_df = pd.read_csv(
    '../../data/segmentationDDH/val_aug_segmentation_dataset.csv')

train_img_list = train_df['file name'].to_list()
train_label_list = train_df['standard mask'].to_list()
train_case_list = train_df['case'].to_list()
train_img_path = '../../data/segmentationDDH/aug_dataset/train/'
val_img_list = val_df['file name'].to_list()
val_label_list = val_df['standard mask'].to_list()
val_case_list = val_df['case'].to_list()
val_img_path = '../../data/segmentationDDH/aug_dataset/val/'

val_image=np.zeros((len(val_img_list),IMAGE_SIZE,IMAGE_SIZE,3),dtype=np.float32)
val_mask=np.zeros((len(val_img_list),IMAGE_SIZE,IMAGE_SIZE,4),dtype=np.float32)    
train_image=np.zeros((len(train_img_list),IMAGE_SIZE,IMAGE_SIZE,3),dtype=np.float32)
train_mask=np.zeros((len(train_img_list),IMAGE_SIZE,IMAGE_SIZE,4),dtype=np.float32)

for i in tqdm(range(len(train_img_list))):
    train_image[i] = np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/image/'+train_img_list[i]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,0]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/1'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,1]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/2'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,2]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/3'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    train_mask[i,:,:,3]=np.array(Image.open(
        train_img_path+str(train_case_list[i])+'/mask/'+str(train_label_list[i]).zfill(5)+'/4'+train_img_list[i][train_img_list[i].find('_'):]).resize((IMAGE_SIZE, IMAGE_SIZE)))

    

for i in tqdm(range(len(val_img_list))):
    val_image[i] = np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/image/'+val_img_list[i]).resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,0]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/1.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,1]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/2.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,2]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/3.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
    val_mask[i,:,:,3]=np.array(Image.open(
        val_img_path+str(val_case_list[i])+'/mask/'+str(val_label_list[i]).zfill(5)+'/4.png').resize((IMAGE_SIZE, IMAGE_SIZE)))
train_image=train_image/255.0
val_image=val_image/255.0
train_mask=train_mask/255.0
val_mask=val_mask/255.0


  0%|          | 0/20088 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

In [5]:

def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return ops.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
        interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    preprocessed = keras.applications.resnet50.preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=preprocessed
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


def dice_loss(pred, target, num_classes=4):
    smooth = 1.
    dice_per_class = tf.Variable(tf.zeros(num_classes))
    
    for class_id in range(num_classes):
        pred_class = pred[:, :,:, class_id]
        target_class = target[:, :,:, class_id]

        intersection = tf.reduce_sum(pred_class * target_class)
        A_sum = tf.reduce_sum(pred_class * pred_class)
        B_sum = tf.reduce_sum(target_class * target_class)

        dice_per_class[class_id].assign(1 - (2. * intersection + smooth) / (A_sum + B_sum + smooth))
    with tf.GradientTape():
        return tf.reduce_mean(dice_per_class)

def dice_cof(pred, target):
    with tf.GradientTape():
        return 1-dice_loss(pred, target, num_classes=4)


In [6]:


with tf.GradientTape():
    model=DeeplabV3Plus(IMAGE_SIZE,NUM_CLASSES)
    checkpoint_filepath = "../../model/segmentation/DeepLabV3_tf_checkpoints.keras"
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only= True
    )
    model.compile(loss=dice_loss, optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=[dice_cof])

    history = model.fit(
        train_image, train_mask, batch_size=BATCH_SIZE, epochs=100, validation_data=(val_image, val_mask),callbacks=[model_checkpoint_callback]
    )



2024-02-26 09:08:31.857794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38158 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


Epoch 1/100


ValueError: No gradients provided for any variable.

In [7]:
dice_loss

<function __main__.dice_loss(pred, target, num_classes=4)>

In [ ]:
train_image.shape